拟牛顿方法不计算海瑟矩阵$\nabla^2f(x)$, 而是构造其近似矩阵$B^k$或者其逆的近似矩阵$H^k$. 我们希望$B^k$或者$H^k$保留了一些海瑟矩阵的性质, 例如使得$d^k$仍然为下降方向.

### 割线方程
设$f(x)$是二次可微的, 向量值函数$\nabla f(x)$在点$x^{k+1}$处的近似为：
$$\nabla f(x)=\nabla f(x^{k+1})+\nabla^2f(x^{k+1})(x-x^{k+1})+O(||x-x^{k+1}||^2)$$
令 $x = x^k, s^k = x^{k+1} − x^k$ 及 $y^k = \nabla f(x^{k+1})− \nabla f(x^k)$, 得到
$$\nabla^2f(x^{k+1})s^k+O(||s^k||^2)=y^k$$
不考虑高阶项的时候, 我们可以得到海瑟矩阵的近似$B^{k+1}$满足：
$$y^{k}=B^{k+1}s^{k}\tag{1}$$
或者其逆矩阵的近似矩阵$H^{k+1}$:
$$s^{k}=H^{k+1}y^k\tag{2}$$
我们得到的近似矩阵$B^k$仍然需要保证它是正定的！对其两边同时左乘$(s^k)^T$得到$(s^k)^Ty^{k}=(s^k)^TB^{k+1}s^{k}$, 从而有条件:
$$(s^k)^Ty^{k}>0\tag{3}$$
对于一般的目标函数, 采取Wolfe准则就会保证曲率条件(3)成立！理由：
$$\nabla f(x^{k+1})^Ts^k\geq c_2\nabla f(x^k)^Ts^k\\
(y^k)^Ts^k\geq(c_2-1)\nabla f(x^{k})^Ts^k>0$$
这是因为$c_2<1$并且$s^k=\alpha_kd^k$是下降方向.

### 一般框架 
![](1.png)

### BFGS 公式
我们这样计算近似矩阵,设$B^{k+1}=B^{k}+auu^{T}+bvv^T$, 其中$a,b\in R$待定, 根据割线方程(1):
$$B^{k+1}s^{k}=y^k=(B^{k}+auu^{T}+bvv^T)s^k$$
整理一下就有：$$(au^Ts^k)u+(bv^Ts^T)v=y^k-B^ks^k$$
直接让两边系数相等就可以得到$u,v,a,b$的公式了, 然后就可以得到$B^k$到$B^{k+1}$的更新公式
$$B^{k+1}=B^{k}+\frac{y^{k}(y^{k})^T}{(s^k)^Ty^k}-\frac{B^ks^k(B^ks^k)^T}{(s^k)^TB^ks^k}.\tag{4}$$
在问题求解过程中，条件(3) 不一定会得到满足，此时应该使用 Wolfe 准则的线搜索来迫使条件(3)成立. 我们根据这个式子并且假设$H^k=(B^k)^{-1}$可以得到：
$$H^{k+1}=(1-\rho_ky^k(s^k)^T)^TH^k(I-\rho_ky^k(s^k)^T)+\rho_ks^k(s^k)^T\tag{5}$$
其中$\rho_k=\frac{1}{(s^k)^Ty^k}$

### 有限内存的BFGS公式
首先我们引入新的记号去表示$H^{k}$的更新公式：
$$H^{k+1}=(V^k)^TH^kV^k+\rho_ks^k(s^k)^T,\tag{6}$$
其中
$$\rho_k=\frac{1}{(y^k)^Ts^k},\quad V^k=I-\rho_ky^k(s^k)^T$$
更新(6)式可以发现它有递归的特点, 于是我们可以展开$m$次

![](2.png)

很明显$H^{k-m}$还是无法显示求出来, 这个式子也不能无限展开. 我们只用一个近似的$H^{k-m}$去近似$H^{k-m}$, 这样这个式子就方便计算了. 这里给出一个算法.

![](3.png)